In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import folium
import datetime
import numpy as np
import pandas as pd
import re
from pytz import timezone
import pytz

In [2]:
def decode_lat_lon(sr: pd.Series) -> pd.Series:
    """
    緯度経度を度数に変換
    """
    tmp = sr.astype(float)
    return (tmp / 100).astype(int) + (tmp % 100) / 60

In [3]:
LOG = "matsumoto-nagano-100318.log"

gps_table = []

with open(LOG) as f:
    for line in f:
        str_list = line.split(",")
        if str_list[0] == "$GPGGA":
            gpgga_list = str_list[1:-1]
        elif str_list[0] == "$GPRMC":
            gprmc_list = str_list[1:-3]
            if gpgga_list[0] != gprmc_list[0]:
                print("なんかおかしい")
                continue
            elif gpgga_list[1:5] != gprmc_list[2:6]:
                print("なんかおかしい")
                continue
            # print(gpgga_list)
            # print(gprmc_list)
            gps_table.append(gprmc_list + gpgga_list[5:])

gps_df = pd.DataFrame(gps_table, columns=[
    "time", "status", "lat", "NS", "lon", "EW",
    "knots", "direction", "date", "quality", "sat",
    "harr", "elevation", "em", "geoid", "gm", "drp_id"
])

gps_df["lat_deg"] = decode_lat_lon(gps_df["lat"])
gps_df["lon_deg"] = decode_lat_lon(gps_df["lon"])
gps_df["lat"] = gps_df["lat"] * ((gps_df.pop("NS") == "N") * 2 - 1)
gps_df["lon"] = gps_df["lon"] * ((gps_df.pop("EW") == "E") * 2 - 1)

gps_df["lat"] = np.deg2rad(gps_df["lat_deg"])
gps_df["lon"] = np.deg2rad(gps_df["lon_deg"])

gps_df["sat"] = gps_df["sat"].astype(int)
gps_df["knots"] = gps_df["knots"].astype(float)
gps_df["direction"] = gps_df["direction"].astype(float)
gps_df["harr"] = gps_df["harr"].astype(float)
gps_df["elevation"] = gps_df["elevation"].astype(float)
gps_df["geoid"] = gps_df["geoid"].astype(float)
gps_df["date"] = (gps_df["date"] + gps_df["time"]).apply(lambda x: pytz.timezone("UTC").localize(
    datetime.datetime.strptime(x, "%d%m%y%H%M%S")).astimezone(timezone("Asia/Tokyo")))

gps_df = gps_df.drop(["em", "gm", "time"], axis=1)

gps_df

,status,lat,lon,knots,direction,date,quality,sat,harr,elevation,geoid,drp_id,lat_deg,lon_deg
0,A,0.632456,2.408442,21.3,276.9,2010-03-18 15:19:59+09:00,1,4,2.2,613.6,37.4,00,36.237045,137.993558
1,A,0.632458,2.408419,21.8,276.3,2010-03-18 15:20:09+09:00,1,4,2.2,614.3,37.4,00,36.237163,137.992265
2,A,0.632466,2.408394,0.3,254.1,2010-03-18 15:20:28+09:00,1,4,2.2,609.0,37.4,00,36.237625,137.990825
3,A,0.632465,2.408396,0.8,102.0,2010-03-18 15:20:43+09:00,1,4,2.2,604.8,37.4,00,36.237567,137.990913
4,A,0.632462,2.408397,6.5,270.7,2010-03-18 15:20:58+09:00,1,4,2.2,579.1,37.4,00,36.237382,137.990982
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
290,A,0.637887,2.410838,5.8,74.3,2010-03-18 16:40:53+09:00,1,7,1.4,384.2,37.4,00,36.548237,138.130830
291,A,0.637902,2.410859,22.4,60.2,2010-03-18 16:41:08+09:00,1,9,1.2,383.0,37.4,00,36.549078,138.132042
292,A,0.637916,2.410894,26.8,54.8,2010-03-18 16:41:23+09:00,1,8,1.2,382.9,37.4,00,36.549897,138.134037
293,A,0.637945,2.410919,31.2,30.5,2010-03-18 16:41:38+09:00,1,8,1.7,381.0,37.4,00,36.551557,138.135477


In [6]:
fig = folium.Figure()

center_lat = 36.45
center_lon = 138.0

m = folium.Map([center_lat, center_lon], zoom_start=10).add_to(fig)
for i in gps_df.index:
    folium.Marker(location=[gps_df["lat_deg"][i], gps_df["lon_deg"][i]]).add_to(m)
m